In [1]:
from elasticsearch import Elasticsearch, helpers
import csv

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Upload to Elasticsearch").setMaster("spark://25.15.27.228:7077")
sc = SparkContext.getOrCreate(conf=conf)
print('hello')

hello


In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Upload to Elasticsearch').getOrCreate()

In [6]:
class ProcssProduct:
    def __init__(self,sc,spark,type = 'Product'):
        self.sc = sc
        self.type = type
        self.spark = spark
        self.URI           = self.sc._gateway.jvm.java.net.URI
        self.Path          = self.sc._gateway.jvm.org.apache.hadoop.fs.Path
        self.FileSystem    = self.sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
        self.Configuration = self.sc._gateway.jvm.org.apache.hadoop.conf.Configuration
        self.colsProduct = ['historical_sold','raw_discount', 'attributes','shop_location','show_free_shipping', 'name','cmt_count', 'price', 'item_rating','show_shopee_verified_label','liked_count']
        self.es = Elasticsearch(host = "25.19.134.189", port = 9200,timeout=30,max_retries=10, retry_on_timeout=True)

        self.getList()
    def getList(self):
        fs = self.FileSystem.get(self.URI("hdfs://cris:9000"), self.Configuration())
        status = fs.listStatus(self.Path(f'/shopee/{self.type}'))
        pathProducts = []
        for fileStatus in status:
            pathProducts.append(str(fileStatus.getPath()))
        self.path = pathProducts[1:]

    def processProduct(self,data):
        data['price'] = data['price'].astype(float)/100000
        data['rating_star'] = data.item_rating.apply(lambda x: eval(x)['rating_star'])
        data.drop(columns=['attributes','item_rating'],inplace=True)
        return data
    def uploadProduct(self):
        for url in self.path:
            parDF2 = self.spark.read.parquet(url)
            tmp = parDF2[self.colsProduct].toPandas()
            data = self.processProduct(tmp)
            try:
                self.uploadHdfs(data)
            except:
                print('Error when upload to HDFS')
            data = self.spark.createDataFrame(data)
            data = data.toJSON().collect()
            try:
                self.uploadES(data)
            except:
                print('Error when upload to ES')

    def uploadHdfs(self,data):
        data = spark.createDataFrame(data.astype(str))
        data.coalesce(1).write.mode('append').parquet(f'hdfs://cris:9000/ProcessShopee/{self.type}/')
        print('Done upload HDFS')
            
    def uploadES(self,data):
        index = self.type.lower()
        i = 1
        for item in data :
            self.es.index(index=index, doc_type='data', id=i, body=item)
            i = i + 1
    
    

In [7]:
product = ProcssProduct(sc,spark,'Product')

In [ ]:
product.uploadProduct()